<a href="https://colab.research.google.com/github/toraaglobal/Case_Studies/blob/master/TheSimpleRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install surprise

     |████████████████████████████████| 3.3MB 2.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/ec/c0/55/3a28eab06b53c220015063ebbdb81213cd3dcbb72c088251ec
Successfully built scikit-surprise


In [0]:
# EXAMPLE BASED OF SURPRISE DOCUMENTATION PAGE
# Additional material available here https://github.com/NicolasHug/Surprise/blob/master/examples/benchmark.py

from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from collections import defaultdict

In [3]:
# OBTAIN

data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()



Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [0]:
# OBTAIN - SCRUB - CREATE TRAINSET
trainset = data.build_full_trainset()


In [5]:
# MODEL

algo = SVD()
algo.fit(trainset)

# cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [6]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9265  0.9291  0.9434  0.9428  0.9381  0.9360  0.0070  
MAE (testset)     0.7326  0.7324  0.7424  0.7448  0.7376  0.7380  0.0050  
Fit time          5.82    5.97    5.94    5.83    5.84    5.88    0.06    
Test time         0.25    0.17    0.24    0.16    0.16    0.19    0.04    


{'fit_time': (5.817701816558838,
  5.967941522598267,
  5.939731121063232,
  5.826546907424927,
  5.8430070877075195),
 'test_mae': array([0.73255194, 0.73242421, 0.74240315, 0.74483388, 0.73756158]),
 'test_rmse': array([0.92653068, 0.9290937 , 0.94336835, 0.94277593, 0.93807153]),
 'test_time': (0.24683427810668945,
  0.16530179977416992,
  0.24304819107055664,
  0.1550452709197998,
  0.15631318092346191)}

In [0]:
# OBTAIN - SCRUB - CREATE TESTSET
testset = trainset.build_anti_testset()


In [0]:
# MODEL - LOOK AT PERFORMANCE ON TEST
predictions = algo.test(testset)


In [0]:
# BUILD FUNCTION FOR GETTING TOP 10 PREDICTIONS
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


In [0]:
# ANALYZE - GET TOP PREDICTIONS

top_n = get_top_n(predictions, n=10)


In [11]:
# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

196 ['408', '318', '199', '64', '169', '316', '272', '529', '483', '178']
186 ['114', '50', '181', '172', '318', '313', '89', '963', '483', '64']
22 ['64', '12', '408', '98', '135', '22', '272', '357', '603', '313']
244 ['23', '187', '285', '127', '175', '483', '511', '199', '654', '134']
166 ['114', '178', '64', '1194', '963', '408', '318', '272', '474', '528']
298 ['408', '64', '169', '12', '513', '528', '661', '316', '176', '302']
115 ['483', '169', '134', '408', '482', '180', '474', '135', '603', '427']
253 ['272', '313', '408', '963', '316', '178', '174', '519', '114', '513']
305 ['132', '23', '124', '603', '114', '515', '514', '57', '657', '661']
6 ['603', '114', '657', '654', '48', '641', '923', '589', '179', '316']
62 ['169', '408', '187', '657', '178', '194', '484', '478', '480', '156']
286 ['318', '923', '64', '12', '187', '205', '603', '178', '489', '131']
200 ['479', '178', '64', '480', '272', '302', '408', '275', '603', '427']
210 ['408', '169', '427', '313', '511', '474',